Q1. Is it OK to initialize all the weights to the same value as long as that value is selected
randomly using He initialization?

Ans. Initializing weights to the same value will create a symmetry in neural network and backpropagation won't be able to change the weights,hence the entire network will behave like a single neuron 

Q2. Is it OK to initialize the bias terms to 0?

Ans. It will make no such difference

Q3.  Name three advantages of the SELU activation function over ReLU.

Ans. 
1. SELU can take negative values,helps in vanishing gradient problem
2. It has a non zero derivative,which avoids the dying units issue
3. Under right conditions,SELU function ensures the model is self normalised

Q4. In which cases would you want to use each of the following activation functions: SELU, leaky ReLU (and its variants), ReLU, tanh, logistic, and softmax?

Ans. 
1. SELU - It is a good default 
2. Leaky ReLU - If you need the network to be as fast as possible, you can use one of the leaky ReLU variants.
3. ReLU - It has the ability to output precisely zero,which can be useful
4. tanh - If the output has to be in between -1 and 1,tanh can be useful
5. logistic - For estimating the probability in the output layer
6. Softmax - To output probabilities for mutually exclusive classes

Q5. What may happen if you set the momentum hyperparameter too close to 1 (e.g., 0.99999) when using an SGD optimizer?

Ans. 

Momentum hyperparameter set to 0 means high friction,1 means no friction,at 0.9999 there will be less friction ,This means that the optimizer will overshoot, then come back, overshoot again, and oscillate like this many times before stabilizing at the minimum. 

Q6. Name three ways you can produce a sparse model.

Ans. 
1. Set tiny weights to zero
2. Apply strong l1 regularizationduring training
3. Use the TF model optimizer toolkit API

Q7. Does dropout slow down training? Does it slow down inference (i.e., making predictions on new instances)? What about MC Dropout?

Ans. 
Dropout slows down the converging process.As per the paper published by Yarin Gal and Zoubin Ghahramani,training a dropout network is mathematically equivalent to aproxximate Bayesian inference.
MC Dropout boosts the dropout model performance.


# Q8. Practice training a deep neural network on the CIFAR10 image dataset:

### a. Build a DNN with 20 hidden layers of 100 neurons each (that’s too many, but it’s the point of this exercise). Use He initialization and the ELU activation function. 

In [5]:
import tensorflow as tf
import keras

In [6]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 activation="elu",
                                 kernel_initializer="he_normal"))

### b. Using Nadam optimization and early stopping, train the network on the CIFAR10 dataset. You can load it with keras.datasets.cifar10.load_​data(). The dataset is composed of 60,000 32 × 32–pixel color images (50,000 for training, 10,000 for testing) with 10 classes, so you’ll need a softmax output layer with 10 neurons. Remember to search for the right learning rate each time you change the model’s architecture or hyperparameters.

In [16]:
from tensorflow import keras

In [12]:
model.add(keras.layers.Dense(10, activation="softmax"))

In [15]:
optimizer = keras.optimizers.Nadam(learning_rate=5e-5)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [17]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()

X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

170508288/170498071 [==============================] - 161s 1us/step


In [19]:
import os

In [20]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [21]:
%tensorboard --logdir=./my_cifar10_logs --port=6006

UsageError: Line magic function `%tensorboard` not found.


In [22]:
model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 16s 9ms/step - loss: 3.7399 - accuracy: 0.1751 - val_loss: 2.1250 - val_accuracy: 0.2300
Epoch 2/100
1407/1407 [==============================] - 11s 8ms/step - loss: 2.0022 - accuracy: 0.2623 - val_loss: 1.9714 - val_accuracy: 0.2730
Epoch 3/100
1407/1407 [==============================] - 13s 9ms/step - loss: 1.8879 - accuracy: 0.3080 - val_loss: 1.8570 - val_accuracy: 0.3244
Epoch 4/100
1407/1407 [==============================] - 17s 12ms/step - loss: 1.8173 - accuracy: 0.3375 - val_loss: 1.7698 - val_accuracy: 0.3454
Epoch 5/100
1407/1407 [==============================] - 13s 9ms/step - loss: 1.7633 - accuracy: 0.3602 - val_loss: 1.7969 - val_accuracy: 0.3442
Epoch 6/100
1407/1407 [==============================] - 12s 9ms/step - loss: 1.7166 - accuracy: 0.3758 - val_loss: 1.7012 - val_accuracy: 0.3800
Epoch 7/100
1407/1407 [==============================] - 12s 9ms/step - loss: 1.6808 - accuracy: 0.3908 - val_loss: 1.7062 

In [23]:
model = keras.models.load_model("my_cifar10_model.h5")
model.evaluate(X_valid, y_valid)

157/157 [==============================] - 0s 2ms/step - loss: 1.4849 - accuracy: 0.4808


[1.4848878383636475, 0.48080000281333923]

### 3. Now try adding Batch Normalization and compare the learning curves: Is it converging faster than before? Does it produce a better model? How does it affect training speed? 

In [25]:
import numpy as np

In [26]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
model.add(keras.layers.BatchNormalization())
for _ in range(20):
    model.add(keras.layers.Dense(100, kernel_initializer="he_normal"))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Activation("elu"))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_bn_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_bn_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

model.fit(X_train, y_train, epochs=100,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

model = keras.models.load_model("my_cifar10_bn_model.h5")
model.evaluate(X_valid, y_valid)

Epoch 1/100
1407/1407 [==============================] - 28s 15ms/step - loss: 1.8461 - accuracy: 0.3368 - val_loss: 1.6690 - val_accuracy: 0.4062
Epoch 2/100
1407/1407 [==============================] - 20s 14ms/step - loss: 1.6716 - accuracy: 0.4047 - val_loss: 1.5697 - val_accuracy: 0.4356
Epoch 3/100
1407/1407 [==============================] - 18s 13ms/step - loss: 1.6027 - accuracy: 0.4304 - val_loss: 1.5443 - val_accuracy: 0.4456
Epoch 4/100
1407/1407 [==============================] - 17s 12ms/step - loss: 1.5513 - accuracy: 0.4468 - val_loss: 1.5078 - val_accuracy: 0.4570
Epoch 5/100
1407/1407 [==============================] - 17s 12ms/step - loss: 1.5060 - accuracy: 0.4658 - val_loss: 1.4446 - val_accuracy: 0.4804
Epoch 6/100
1407/1407 [==============================] - 16s 11ms/step - loss: 1.4709 - accuracy: 0.4799 - val_loss: 1.4118 - val_accuracy: 0.4986
Epoch 7/100
1407/1407 [==============================] - 16s 11ms/step - loss: 1.4348 - accuracy: 0.4886 - val_loss: 1

[1.325295090675354, 0.5360000133514404]

As per the outputs,the model converges faster,has better accuracy and better training speed

### d. Try replacing Batch Normalization with SELU, and make the necessary adjustements to ensure the network self-normalizes (i.e., standardize the input features, use LeCun normal initialization, make sure the DNN contains only a sequence of dense layers, etc.).

In [28]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 kernel_initializer="lecun_normal",
                                 activation="selu"))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(learning_rate=7e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_selu_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_selu_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=callbacks)

model = keras.models.load_model("my_cifar10_selu_model.h5")
model.evaluate(X_valid_scaled, y_valid)

Epoch 1/100
1407/1407 [==============================] - 27s 16ms/step - loss: 1.9028 - accuracy: 0.3216 - val_loss: 1.8474 - val_accuracy: 0.3482
Epoch 2/100
1407/1407 [==============================] - 21s 15ms/step - loss: 1.6984 - accuracy: 0.3999 - val_loss: 1.7947 - val_accuracy: 0.3474
Epoch 3/100
1407/1407 [==============================] - 21s 15ms/step - loss: 1.6019 - accuracy: 0.4346 - val_loss: 1.7130 - val_accuracy: 0.3970
Epoch 4/100
1407/1407 [==============================] - 22s 16ms/step - loss: 1.5365 - accuracy: 0.4613 - val_loss: 1.6063 - val_accuracy: 0.4456
Epoch 5/100
1407/1407 [==============================] - 20s 14ms/step - loss: 1.4802 - accuracy: 0.4825 - val_loss: 1.5612 - val_accuracy: 0.4598
Epoch 6/100
1407/1407 [==============================] - 20s 14ms/step - loss: 1.4378 - accuracy: 0.4985 - val_loss: 1.5162 - val_accuracy: 0.4764
Epoch 7/100
1407/1407 [==============================] - 20s 14ms/step - loss: 1.3913 - accuracy: 0.5130 - val_loss: 1

[1.479697346687317, 0.4885999858379364]

### e. Try regularizing the model with alpha dropout. Then, without retraining your model, see if you can achieve better accuracy using MC Dropout.

In [29]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 kernel_initializer="lecun_normal",
                                 activation="selu"))

model.add(keras.layers.AlphaDropout(rate=0.1))
model.add(keras.layers.Dense(10, activation="softmax"))

optimizer = keras.optimizers.Nadam(learning_rate=5e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_alpha_dropout_model.h5", save_best_only=True)
run_index = 1 # increment every time you train the model
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_alpha_dropout_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

X_means = X_train.mean(axis=0)
X_stds = X_train.std(axis=0)
X_train_scaled = (X_train - X_means) / X_stds
X_valid_scaled = (X_valid - X_means) / X_stds
X_test_scaled = (X_test - X_means) / X_stds

model.fit(X_train_scaled, y_train, epochs=100,
          validation_data=(X_valid_scaled, y_valid),
          callbacks=callbacks)

model = keras.models.load_model("my_cifar10_alpha_dropout_model.h5")
model.evaluate(X_valid_scaled, y_valid)

Epoch 1/100
1407/1407 [==============================] - 28s 16ms/step - loss: 1.8815 - accuracy: 0.3278 - val_loss: 1.7211 - val_accuracy: 0.4020
Epoch 2/100
1407/1407 [==============================] - 21s 15ms/step - loss: 1.6534 - accuracy: 0.4176 - val_loss: 1.6282 - val_accuracy: 0.4244
Epoch 3/100
1407/1407 [==============================] - 22s 15ms/step - loss: 1.5634 - accuracy: 0.4507 - val_loss: 1.6071 - val_accuracy: 0.4334
Epoch 4/100
1407/1407 [==============================] - 22s 16ms/step - loss: 1.5011 - accuracy: 0.4714 - val_loss: 1.5902 - val_accuracy: 0.4504
Epoch 5/100
1407/1407 [==============================] - 22s 15ms/step - loss: 1.4433 - accuracy: 0.4918 - val_loss: 1.5304 - val_accuracy: 0.4636
Epoch 6/100
1407/1407 [==============================] - 21s 15ms/step - loss: 1.3919 - accuracy: 0.5127 - val_loss: 1.5480 - val_accuracy: 0.4864
Epoch 7/100
1407/1407 [==============================] - 21s 15ms/step - loss: 1.3560 - accuracy: 0.5258 - val_loss: 1

[1.4859399795532227, 0.49639999866485596]